In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
import pandas as pd
from torch.utils.data import Dataset
import os
import torchaudio

class My_Dataset(Dataset):

    def __init__(self, csv_file, root_dir, set_number):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the audio files.
            set_number (int): Set number to filter the samples.
        """
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.set_number = set_number
        self.filtered_indices = self.filter()

    def __len__(self):
        return len(self.filtered_indices)

    def filter(self):
        filtered_indices = []
        for idx in range(len(self.data)):
            set_value = int(self.data.iloc[idx, 1])
            if set_value == self.set_number:
                filtered_indices.append(idx)
        return filtered_indices

    def __getitem__(self, idx):
        idx = self.filtered_indices[idx]
        file = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        class_label = int(self.data.iloc[idx, 2])
        waveform, _ = torchaudio.load(file)
        return waveform, class_label

csv_file = 'dataset.csv'
root_dir = 'Inregistrari'
train_dataset = My_Dataset(csv_file, root_dir, 0)
test_dataset = My_Dataset(csv_file, root_dir, 1)
valid_dataset = My_Dataset(csv_file, root_dir, 2)
print(train_dataset[0])
print(test_dataset[0])
print(valid_dataset[0])


(tensor([[-0.0282, -0.0244, -0.0236,  ...,  0.0104,  0.0114,  0.0126]]), 3)
(tensor([[ 9.1553e-05, -2.7466e-04, -9.1553e-04,  ..., -3.9673e-04,
         -4.8828e-04, -4.5776e-04]]), 2)
(tensor([[-0.0032, -0.0017, -0.0019,  ...,  0.0024,  0.0081,  0.0065]]), 3)


In [4]:
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader
# from torchvision import datasets
from torchvision.transforms import ToTensor
plt.style.use('ggplot')

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

for batch, (X, y) in enumerate(train_loader):
    print(X.shape, y.shape)
    if batch == 3:
        break

# Print class distribution
class_counts = train_dataset.data.iloc[train_dataset.filtered_indices, 2].value_counts()
print("Class distribution in training set:", class_counts)

torch.Size([64, 1, 16000]) torch.Size([64])
torch.Size([64, 1, 16000]) torch.Size([64])
torch.Size([64, 1, 16000]) torch.Size([64])
torch.Size([64, 1, 16000]) torch.Size([64])
Class distribution in training set: class
4    60
0    60
2    59
5    57
3    55
1    45
Name: count, dtype: int64


In [5]:
classes = ['start', 'stop', 'home', 'pick_up', 'approach', 'free']

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

class SimplifiedWaveNet(nn.Module):
    def __init__(self, num_classes, num_channels=128, num_layers=3, num_filters=16, dilation_rates=None):
        super(SimplifiedWaveNet, self).__init__()
        
        if dilation_rates is None:
            dilation_rates = [2**i for i in range(num_layers)]
        
        self.num_layers = num_layers
        self.num_filters = num_filters
        self.dilation_rates = dilation_rates
        
        self.init_conv = nn.Conv1d(num_channels, num_filters, kernel_size=1)
        
        self.dilated_layers = nn.ModuleList()
        for dilation_rate in dilation_rates:
            self.dilated_layers.append(nn.Sequential(
                nn.Conv1d(num_filters, num_filters, kernel_size=2, stride=1,
                          dilation=dilation_rate, padding=dilation_rate),
                nn.ReLU(inplace=True)
            ))
        
        self.final_conv = nn.Conv1d(num_filters, num_classes, kernel_size=1)
    
    def forward(self, x):
        x = self.init_conv(x)
        
        skip_connections = []
        for layer in self.dilated_layers:
            x = layer(x)
            skip_connections.append(x)
        
        min_size = min([s.size(2) for s in skip_connections])
        adjusted_skip_connections = [s[:, :, :min_size] for s in skip_connections]
        
        x = sum(adjusted_skip_connections)
        x = F.relu(x)
        x = self.final_conv(x)
        
        x = F.adaptive_avg_pool1d(x, 1).squeeze(-1)
        
        return x

def get_simplified_wavenet(num_classes, num_channels=128, num_layers=3, num_filters=16, dilation_rates=None, device='cpu'):
    model = SimplifiedWaveNet(num_classes, num_channels, num_layers, num_filters, dilation_rates)
    return model.to(device)

# Example usage with the correct number of input channels:
num_input_channels = 128  # Adjust as needed
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = get_simplified_wavenet(num_classes=6, num_channels=num_input_channels, num_layers=3, num_filters=16, device=device)
print(model)

SimplifiedWaveNet(
  (init_conv): Conv1d(128, 16, kernel_size=(1,), stride=(1,))
  (dilated_layers): ModuleList(
    (0): Sequential(
      (0): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(1,))
      (1): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(2,), dilation=(2,))
      (1): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(4,), dilation=(4,))
      (1): ReLU(inplace=True)
    )
  )
  (final_conv): Conv1d(16, 6, kernel_size=(1,), stride=(1,))
)


In [7]:
import  torch.optim as optim
# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Reduced learning rate
criterion = nn.CrossEntropyLoss()

In [8]:
train_losses = []
train_accuracies = []

# Training function
def train(model, device, train_loader, optimizer, epoch, log_interval, scheduler):
    model.train()
    running_loss = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        data = data.squeeze(1)  # Remove channel dimension
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    avg_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / len(train_loader.dataset)
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    scheduler.step()

In [45]:
def number_of_correct(pred, target):
    return pred.squeeze().eq(target).sum().item()

def get_likely_index(tensor):
    return tensor.argmax(dim=-1)

test_losses = []
test_accuracies = []
all_preds = []
all_targets = []

# Testing function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # data = data.squeeze(1)  # Remove channel dimension
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item() * data.size(0)  # Accumulate the loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    avg_test_loss = test_loss / len(test_loader.dataset)  # Calculate average loss
    accuracy = 100. * correct / len(test_loader.dataset)
    test_losses.append(avg_test_loss)
    test_accuracies.append(accuracy)
    print(f'\nTest set: Average loss: {avg_test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')


In [10]:
import torch
from torch import optim
from tqdm import tqdm

# Assuming model, device, train_loader, test_loader, optimizer are defined
# Assuming train and test functions are defined

n_epoch = 150
log_interval = 20

# Example of setting up a scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, device, train_loader, optimizer, epoch, log_interval, scheduler)
        test(model, device, test_loader)
        torch.save(model.state_dict(), 'wavenet_command.pt')
        scheduler.step()  # Update the learning rate
        pbar.update(1)

  0%|          | 0/150 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [16, 128, 1], expected input[1, 64, 16000] to have 128 channels, but got 64 channels instead

In [46]:
import torch
import pandas as pd
from torch.utils.data import Dataset
import os
import torchaudio

class My_Dataset(Dataset):
    def __init__(self, csv_file, root_dir, set_number):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.set_number = set_number
        self.filtered_indices = self.filter()

    def __len__(self):
        return len(self.filtered_indices)

    def filter(self):
        filtered_indices = []
        for idx in range(len(self.data)):
            set_value = int(self.data.iloc[idx, 1])
            if set_value == self.set_number:
                filtered_indices.append(idx)
        return filtered_indices

    def __getitem__(self, idx):
        idx = self.filtered_indices[idx]
        file = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        class_label = int(self.data.iloc[idx, 2])
        waveform, _ = torchaudio.load(file)
        # Normalize the waveform
        waveform = (waveform - waveform.mean()) / waveform.std()
        # if waveform.shape[0] != num_input_channels:
        #     waveform = waveform.repeat(num_input_channels // waveform.shape[0], 1)
        return waveform, class_label

csv_file = 'dataset.csv'
root_dir = 'Inregistrari'
num_input_channels = 128  # Ensure this matches the expected input channels for the model

train_dataset = My_Dataset(csv_file, root_dir, 0)
test_dataset = My_Dataset(csv_file, root_dir, 1)
valid_dataset = My_Dataset(csv_file, root_dir, 2)



import torch.nn as nn
import torch.nn.functional as F

class SimplifiedWaveNet(nn.Module):
    def __init__(self, num_classes, num_channels=1, num_layers=4, num_filters=8, dilation_rates=None):
        super(SimplifiedWaveNet, self).__init__()
        self.num_layers = num_layers
        
        self.init_conv = nn.Conv1d(num_channels, num_filters, kernel_size=1)

        self.conv1 = nn.Conv1d(num_filters, num_filters * 2, kernel_size=3, padding=1, dilation=2)
        self.conv2 = nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size=3, padding=2, dilation=4)
        self.conv3 = nn.Conv1d(num_filters * 4, num_filters * 8, kernel_size=3, padding=4, dilation=8)
        self.conv4 = nn.Conv1d(num_filters * 8, num_filters * 16, kernel_size=3, padding=8, dilation=16)

        self.pool = nn.MaxPool1d(4)

        self.final_conv = nn.Conv1d(256, 128, kernel_size=1)
        self.linear = nn.Linear(128 * 57 * 1, 256)
        self.linear2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.init_conv(x)
        x1 = self.pool(F.relu(self.conv1(x)))
        x2 = self.pool(F.relu(self.conv2(x1)))
        x3 = self.pool(F.relu(self.conv3(x2)))
        x4 = self.pool(F.relu(self.conv4(x3)))
        x5 = self.final_conv(x4)
        # print(x4.shape)
        x5 = x5.view(x5.size(0), -1)
        x6 = F.relu(self.linear(x5))
        x7 = self.linear2(x6)
        return x7



def get_simplified_wavenet(num_classes, num_channels=1, num_layers=3, num_filters=16, dilation_rates=None, device='cpu'):
    model = SimplifiedWaveNet(num_classes, num_channels, num_layers, num_filters, dilation_rates)
    return model.to(device)

model=get_simplified_wavenet(6)



#print the number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

nr = count_parameters(model)
print(f'The model has {nr:,} trainable parameters')
num_input_channels = 128  # Adju


import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    

optimizer = optim.Adam(model.parameters(), lr=0.001)  # Increased learning rate
criterion = nn.CrossEntropyLoss()

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
all_preds = []
all_targets = []

def train(model, device, train_loader, optimizer, epoch, log_interval, scheduler):
    model.train()
    running_loss = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # data = data.squeeze(1)  # Remove channel dimension

        # print(data.shape, target.shape)


        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    avg_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / len(train_loader.dataset)
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    scheduler.step()
    print(f'Train Epoch: {epoch}\tAverage Loss: {avg_loss:.6f}\tAccuracy: {accuracy:.2f}%')


n_epoch = 150
log_interval = 20

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, device, train_loader, optimizer, epoch, log_interval, scheduler)
        test(model, device, test_loader)
        torch.save(model.state_dict(), 'wavenet_command.pt')
        scheduler.step()  # Update the learning rate
        pbar.update(1)



The model has 2,033,542 trainable parameters


  0%|          | 0/150 [00:00<?, ?it/s]

Train Epoch: 1 [0/336 (0%)]	Loss: 1.795812
Train Epoch: 1	Average Loss: 1.813363	Accuracy: 14.58%


  1%|          | 1/150 [00:03<09:51,  3.97s/it]


Test set: Average loss: 1.8005, Accuracy: 12/96 (12%)

Train Epoch: 2 [0/336 (0%)]	Loss: 1.776709
Train Epoch: 2	Average Loss: 1.783729	Accuracy: 17.26%


...


Test set: Average loss: 0.8023, Accuracy: 78/96 (81%)

Train Epoch: 149 [0/336 (0%)]	Loss: 0.072655
Train Epoch: 149	Average Loss: 0.052845	Accuracy: 100.00%


 99%|█████████▉| 149/150 [10:58<00:04,  4.49s/it]


Test set: Average loss: 0.8023, Accuracy: 78/96 (81%)

Train Epoch: 150 [0/336 (0%)]	Loss: 0.050512
Train Epoch: 150	Average Loss: 0.054352	Accuracy: 100.00%


100%|██████████| 150/150 [11:03<00:00,  4.42s/it]


Test set: Average loss: 0.8023, Accuracy: 78/96 (81%)



In [47]:
from sklearn.metrics import classification_report

# Assuming the testing loop code here

# After the testing loop
report = classification_report(all_targets, all_preds, target_names=['start', 'stop', 'home', 'pick_up', 'approach', 'free'])  # Adjust target_names based on your dataset
print(report)
from sklearn.metrics import confusion_matrix

confision_matrix = confusion_matrix(all_targets, all_preds)
print(confision_matrix)

              precision    recall  f1-score   support

       start       0.75      0.88      0.81      1800
        stop       0.70      0.61      0.65      3300
        home       0.71      0.71      0.71      2250
     pick_up       0.86      0.76      0.81      2850
    approach       0.79      0.99      0.88      2100
        free       0.94      0.89      0.92      2100

    accuracy                           0.79     14400
   macro avg       0.79      0.81      0.79     14400
weighted avg       0.79      0.79      0.78     14400

[[1584  148    7   37    7   17]
 [ 307 2007  469  309  182   26]
 [ 163  430 1598    6   16   37]
 [  33  293   10 2173  314   27]
 [   6    2    9    0 2070   13]
 [  19    4  163    0   37 1877]]


In [51]:
class SimplifiedWaveNet(nn.Module):
    def __init__(self, num_classes, num_channels=1, num_layers=4, num_filters=8, dilation_rates=None):
        super(SimplifiedWaveNet, self).__init__()
        self.num_layers = num_layers
        
        self.init_conv = nn.Conv1d(num_channels, num_filters, kernel_size=1)

        self.conv1 = nn.Conv1d(num_filters, num_filters * 2, kernel_size=3, padding=1, dilation=2)
        self.conv2 = nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size=3, padding=2, dilation=4)
        self.conv3 = nn.Conv1d(num_filters * 4, num_filters * 8, kernel_size=3, padding=4, dilation=8)
        self.conv4 = nn.Conv1d(num_filters * 8, num_filters * 16, kernel_size=3, padding=8, dilation=16)

        self.pool = nn.MaxPool1d(4)

        self.final_conv = nn.Conv1d(256, 128, kernel_size=1)
        self.linear = nn.Linear(128 * 57 * 1, 256)
        self.linear2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.init_conv(x)
        x1 = self.pool(F.relu(self.conv1(x)))
        x2 = self.pool(F.relu(self.conv2(x1)))
        x3 = self.pool(F.relu(self.conv3(x2)))
        x4 = self.pool(F.relu(self.conv4(x3)))
        x5 = self.final_conv(x4)
        # print(x4.shape)
        x5 = x5.view(x5.size(0), -1)
        x6 = F.relu(self.linear(x5))
        x7 = self.linear2(x6)
        return x7



def get_simplified_wavenet(num_classes, num_channels=1, num_layers=3, num_filters=16, dilation_rates=None, device='cpu'):
    model = SimplifiedWaveNet(num_classes, num_channels, num_layers, num_filters, dilation_rates)
    return model.to(device)

model=get_simplified_wavenet(6)
print(model)

SimplifiedWaveNet(
  (init_conv): Conv1d(1, 16, kernel_size=(1,), stride=(1,))
  (conv1): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,), dilation=(2,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(4,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(8,))
  (conv4): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(16,))
  (pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (final_conv): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
  (linear): Linear(in_features=7296, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=6, bias=True)
)


In [48]:
import torch
import pandas as pd
from torch.utils.data import Dataset
import os
import torchaudio

class My_Dataset(Dataset):
    def __init__(self, csv_file, root_dir, set_number):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.set_number = set_number
        self.filtered_indices = self.filter()

    def __len__(self):
        return len(self.filtered_indices)

    def filter(self):
        filtered_indices = []
        for idx in range(len(self.data)):
            set_value = int(self.data.iloc[idx, 1])
            if set_value == self.set_number:
                filtered_indices.append(idx)
        return filtered_indices

    def __getitem__(self, idx):
        idx = self.filtered_indices[idx]
        file = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        # class_label = int(self.data.iloc[idx, 2])
        speaker_id = int(self.data.iloc[idx, 3])
        waveform, _ = torchaudio.load(file)
        # Normalize the waveform
        waveform = (waveform - waveform.mean()) / waveform.std()
        # if waveform.shape[0] != num_input_channels:
        #     waveform = waveform.repeat(num_input_channels // waveform.shape[0], 1)
        return waveform, speaker_id

csv_file = 'dataset.csv'
root_dir = 'Inregistrari'
num_input_channels = 128  # Ensure this matches the expected input channels for the model

train_dataset = My_Dataset(csv_file, root_dir, 0)
test_dataset = My_Dataset(csv_file, root_dir, 1)
valid_dataset = My_Dataset(csv_file, root_dir, 2)



import torch.nn as nn
import torch.nn.functional as F

class SimplifiedWaveNet(nn.Module):
    def __init__(self, num_classes, num_channels=1, num_layers=4, num_filters=8, dilation_rates=None):
        super(SimplifiedWaveNet, self).__init__()
        self.num_layers = num_layers
        
        self.init_conv = nn.Conv1d(num_channels, num_filters, kernel_size=1)

        self.conv1 = nn.Conv1d(num_filters, num_filters * 2, kernel_size=3, padding=1, dilation=2)
        self.conv2 = nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size=3, padding=2, dilation=4)
        self.conv3 = nn.Conv1d(num_filters * 4, num_filters * 8, kernel_size=3, padding=4, dilation=8)
        self.conv4 = nn.Conv1d(num_filters * 8, num_filters * 16, kernel_size=3, padding=8, dilation=16)

        self.pool = nn.MaxPool1d(4)

        self.final_conv = nn.Conv1d(256, 128, kernel_size=1)
        self.linear = nn.Linear(128 * 57 * 1, 256)
        self.linear2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.init_conv(x)
        x1 = self.pool(F.relu(self.conv1(x)))
        x2 = self.pool(F.relu(self.conv2(x1)))
        x3 = self.pool(F.relu(self.conv3(x2)))
        x4 = self.pool(F.relu(self.conv4(x3)))
        x5 = self.final_conv(x4)
        # print(x4.shape)
        x5 = x5.view(x5.size(0), -1)
        x6 = F.relu(self.linear(x5))
        x7 = self.linear2(x6)
        return x7



def get_simplified_wavenet(num_classes, num_channels=1, num_layers=3, num_filters=16, dilation_rates=None, device='cpu'):
    model = SimplifiedWaveNet(num_classes, num_channels, num_layers, num_filters, dilation_rates)
    return model.to(device)

model=get_simplified_wavenet(8)



#print the number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

nr = count_parameters(model)
print(f'The model has {nr:,} trainable parameters')
num_input_channels = 128  # Adju


import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    

optimizer = optim.Adam(model.parameters(), lr=0.001)  # Increased learning rate
criterion = nn.CrossEntropyLoss()

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
all_preds = []
all_targets = []

def train(model, device, train_loader, optimizer, epoch, log_interval, scheduler):
    model.train()
    running_loss = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # data = data.squeeze(1)  # Remove channel dimension

        # print(data.shape, target.shape)


        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    avg_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / len(train_loader.dataset)
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    scheduler.step()
    print(f'Train Epoch: {epoch}\tAverage Loss: {avg_loss:.6f}\tAccuracy: {accuracy:.2f}%')


n_epoch = 150
log_interval = 20

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, device, train_loader, optimizer, epoch, log_interval, scheduler)
        test(model, device, test_loader)
        torch.save(model.state_dict(), 'wavenet_command.pt')
        scheduler.step()  # Update the learning rate
        pbar.update(1)



Train Epoch: 138 [0/336 (0%)]	Loss: 0.237979
Train Epoch: 138	Average Loss: 0.178298	Accuracy: 96.13%


 92%|█████████▏| 138/150 [10:27<00:52,  4.34s/it]


Test set: Average loss: 0.9582, Accuracy: 75/96 (78%)

Train Epoch: 139 [0/336 (0%)]	Loss: 0.171575
Train Epoch: 139	Average Loss: 0.180616	Accuracy: 96.13%


 93%|█████████▎| 139/150 [10:31<00:49,  4.50s/it]


Test set: Average loss: 0.9582, Accuracy: 75/96 (78%)

Train Epoch: 140 [0/336 (0%)]	Loss: 0.168538
Train Epoch: 140	Average Loss: 0.171533	Accuracy: 96.13%


...


Test set: Average loss: 0.9582, Accuracy: 75/96 (78%)

Train Epoch: 149 [0/336 (0%)]	Loss: 0.163532
Train Epoch: 149	Average Loss: 0.172698	Accuracy: 96.13%


 99%|█████████▉| 149/150 [11:22<00:05,  5.00s/it]


Test set: Average loss: 0.9582, Accuracy: 75/96 (78%)

Train Epoch: 150 [0/336 (0%)]	Loss: 0.130939
Train Epoch: 150	Average Loss: 0.202732	Accuracy: 96.13%


100%|██████████| 150/150 [11:26<00:00,  4.58s/it]


Test set: Average loss: 0.9582, Accuracy: 75/96 (78%)



In [50]:
from sklearn.metrics import classification_report

# Assuming the testing loop code here

# After the testing loop
report = classification_report(all_targets, all_preds, target_names=['Anca', 'Adi', 'Bobo', 'Danila', 'Mada', 'Toni', 'Oana', 'Luci'])  # Adjust target_names based on your dataset
print(report)
from sklearn.metrics import confusion_matrix

confision_matrix = confusion_matrix(all_targets, all_preds)
print(confision_matrix)

              precision    recall  f1-score   support

        Anca       0.76      0.96      0.85      1350
         Adi       0.85      0.87      0.86      1500
        Bobo       0.83      0.87      0.85      1800
      Danila       0.66      0.63      0.65      2250
        Mada       0.75      0.72      0.73      1650
        Toni       0.94      0.73      0.82      1950
        Oana       0.69      0.64      0.66      1800
        Luci       0.58      0.65      0.61      2100

    accuracy                           0.74     14400
   macro avg       0.76      0.76      0.75     14400
weighted avg       0.75      0.74      0.74     14400

[[1293    7    5   22    1   16    6    0]
 [  19 1301    0   16    0   28    0  136]
 [   6  158 1571   13   42    1    9    0]
 [  32   14    7 1428  188   14   41  526]
 [   7    1  164  150 1182    0  146    0]
 [ 176   14  127  175    0 1427    8   23]
 [  11    1    9  183  140   15 1153  288]
 [ 155   40    8  184   16   24  313 1360]]
